In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-i-sedelkin/data/RFM_ht_data.csv', parse_dates=['InvoiceDate'])

/opt/tljh/user/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
data[['InvoiceNo', 'CustomerCode']] = data[['InvoiceNo', 'CustomerCode']].astype('string')

In [24]:
data.dtypes

InvoiceNo               string
CustomerCode            string
InvoiceDate     datetime64[ns]
Amount                 float64
dtype: object

In [31]:
last_date = data.InvoiceDate.max()

In [100]:
data.CustomerCode.nunique()

123733

In [56]:
rfm_values = data.groupby('CustomerCode').agg({'InvoiceDate': lambda x: (last_date - x.max()).days, \
                                              'InvoiceNo': lambda x : len(x), \
                                              'Amount': lambda x: sum(x)})

In [57]:
rfm_values.rename(columns={'InvoiceDate': 'recency', 'InvoiceNo': 'frequency', 'Amount': 'monetary'}, inplace=True)

In [60]:
rfm_values.head()

,recency,frequency,monetary
CustomerCode,,,
02213019,19,1,1609.20
02213042,22,3,9685.48
02213071,29,1,415.00
02213088,23,1,305.00
02213092,25,1,1412.88


In [58]:
quantiles = rfm_values.quantile([0.25, 0.5, 0.75])
quantiles

,recency,frequency,monetary
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [85]:
rfm_table = rfm_values

In [105]:
def r_func(value, quantiles, parameter): 
    if value <= quantiles[parameter][0.25]: return 1
    elif value <= quantiles[parameter][0.50]: return 2
    elif value <= quantiles[parameter][0.75]: return 3
    else: return 4


def fm_func(value, quantiles, parameter): 
    if value <= quantiles[parameter][0.25]: return 4
    elif value <= quantiles[parameter][0.50]: return 3
    elif value <= quantiles[parameter][0.75]: return 2
    else: return 1

In [106]:
rfm_table['r_segment'] = rfm_values['recency'].apply(lambda x: r_func(x, quantiles, 'recency'))
rfm_table['f_segment'] = rfm_values['frequency'].apply(lambda x: fm_func(x, quantiles, 'frequency'))
rfm_table['m_segment'] = rfm_values['monetary'].apply(lambda x: fm_func(x, quantiles, 'monetary'))

In [107]:
rfm_table['rfm_segment'] = rfm_table['r_segment'].astype('string') + rfm_table['f_segment'].astype('string') + rfm_table['m_segment'].astype('string')
rfm_table.head()

,recency,frequency,monetary,r_segment,f_segment,m_segment,rfm_segment
CustomerCode,,,,,,,
02213019,19,1,1609.20,4,4,3,443
02213042,22,3,9685.48,4,2,1,421
02213071,29,1,415.00,4,4,4,444
02213088,23,1,305.00,4,4,4,444
02213092,25,1,1412.88,4,4,3,443


In [104]:
rfm_table.shape

(123733, 7)

In [126]:
rfm_table.groupby('rfm_segment', as_index=False).agg(members=('recency', 'count')).sort_values(by='members', ascending=False)

,rfm_segment,members
63,444,10624
0,111,9705
62,443,6729
47,344,6593
16,211,5847
...,...,...
55,424,63
19,214,60
3,114,60
35,314,33


In [ ]:
rfm_pivot = rf